In [1]:
import re

def generate_plural(word):
    if re.search(r'(s|x|z|ch|sh)$', word):
        return word + 'es', 'E-insertion'
    elif re.search(r'[^aeiou]y$', word):
        return word[:-1] + 'ies', 'Y-replacement'
    else:
        return word + 's', 'S-addition'


def analyze_word(word, known_nouns):
    if word in known_nouns:
        return f"{word}+N+SG"

    # Try rule 1: E-insertion
    if word.endswith("es"):
        stem = word[:-2]
        if re.search(r'(s|x|z|ch|sh)$', stem) and stem in known_nouns:
            return f"{stem}+N+PL"

    # Try rule 2: Y-replacement
    if word.endswith("ies"):
        stem = word[:-3] + "y"
        if stem in known_nouns and re.search(r'[^aeiou]y$', stem):
            return f"{stem}+N+PL"

    # Try rule 3: S-addition
    if word.endswith("s"):
        stem = word[:-1]
        if stem in known_nouns and not re.search(r'(s|x|z|ch|sh|[^aeiou]y)$', stem):
            return f"{stem}+N+PL"

    return "Invalid Word"


# Load known nouns (assume singulars)
with open("brown_nouns.txt", "r") as f:
    known_nouns = set(word.strip().lower() for word in f)

# Example input list (replace with actual test)
test_words = ["foxes", "fox", "tries", "try", "bags", "bag", "foxs", "trys", "watchs"]

for word in test_words:
    print(f"{word} = {analyze_word(word.lower(), known_nouns)}")


foxes = fox+N+PL
fox = fox+N+SG
tries = tries+N+SG
try = try+N+SG
bags = bags+N+SG
bag = bag+N+SG
foxs = Invalid Word
trys = Invalid Word
watchs = Invalid Word


In [2]:
# FST for English noun pluralization

def pluralize(noun):
    # Define endings that require 'es'
    es_endings = ('s', 'z', 'x', 'ch', 'sh')
    # Check for singular
    if noun.endswith('s') or noun.endswith('es'):
        if noun.endswith('es'):
            stem = noun[:-2]
            if stem.endswith(es_endings):
                return f"{stem}+N+PL"
            else:
                return "Invalid Word"
        else:
            return "Invalid Word"
    elif noun.endswith('ies'):
        # Y replacement
        stem = noun[:-3]
        if stem.endswith('y'):
            return f"{stem}+N+PL"
        else:
            return "Invalid Word"
    elif noun.endswith('s'):
        # Regular plural
        stem = noun[:-1]
        if stem.endswith(es_endings):
            # Should be 'es' not just 's'
            return "Invalid Word"
        elif stem.endswith('y'):
            # Should be 'ies' not just 's'
            return "Invalid Word"
        else:
            return f"{stem}+N+PL"
    else:
        # Singular noun
        return f"{noun}+N+SG"

# Examples
words = ["fox", "foxes", "foxs", "watch", "watches", "bag", "bags", "try", "tries", "trys"]

for word in words:
    print(f"{word} = {pluralize(word)}")

fox = fox+N+SG
foxes = fox+N+PL
foxs = Invalid Word
watch = watch+N+SG
watches = watch+N+PL
bag = bag+N+SG
bags = Invalid Word
try = try+N+SG
tries = Invalid Word
trys = Invalid Word


In [3]:
# FST for English noun pluralization (correct Y-replacement handling)

def pluralize(noun):
    es_endings = ('s', 'z', 'x', 'ch', 'sh')

    # Plural: "ies" ending (Y-replacement rule)
    if noun.endswith('ies'):
        stem = noun[:-3]
        # The singular should be stem + "y"
        return f"{stem}y+N+PL"

    # Plural: "es" ending (E-insertion rule)
    elif noun.endswith('es'):
        stem = noun[:-2]
        # If the stem ends with an ES-ending, it's valid
        if stem.endswith(es_endings):
            return f"{stem}+N+PL"
        else:
            return "Invalid Word"

    # Plural: regular "s" ending (S addition rule)
    elif noun.endswith('s'):
        stem = noun[:-1]
        # If the stem ends with an ES-ending, it should have 'es', not just 's'
        if stem.endswith(es_endings):
            return "Invalid Word"
        # If the stem ends with 'y', it should have 'ies', not just 's'
        if stem.endswith('y'):
            return "Invalid Word"
        return f"{stem}+N+PL"

    # Singular noun
    else:
        return f"{noun}+N+SG"

# Examples
words = ["fox", "foxes", "foxs", "watch", "watches", "bag", "bags", "try", "tries", "trys"]

for word in words:
    print(f"{word} = {pluralize(word)}")

fox = fox+N+SG
foxes = fox+N+PL
foxs = Invalid Word
watch = watch+N+SG
watches = watch+N+PL
bag = bag+N+SG
bags = bag+N+PL
try = try+N+SG
tries = try+N+PL
trys = Invalid Word


In [5]:
# Pluralizer for all nouns in brown_noun.txt

def pluralize(noun):
    es_endings = ('s', 'z', 'x', 'ch', 'sh')

    # Plural: "ies" ending (Y-replacement rule)
    if noun.endswith('ies'):
        stem = noun[:-3]
        return f"{stem}y+N+PL"

    # Plural: "es" ending (E-insertion rule)
    elif noun.endswith('es'):
        stem = noun[:-2]
        if stem.endswith(es_endings):
            return f"{stem}+N+PL"
        else:
            return "Invalid Word"

    # Plural: regular "s" ending (S addition rule)
    elif noun.endswith('s'):
        stem = noun[:-1]
        if stem.endswith(es_endings):
            return "Invalid Word"
        if stem.endswith('y'):
            return "Invalid Word"
        return f"{stem}+N+PL"

    # Singular noun
    else:
        return f"{noun}+N+SG"


def process_file(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            noun = line.strip()
            if not noun:
                continue
            result = pluralize(noun)
            outfile.write(f"{noun} = {result}\n")


if __name__ == "__main__":
    process_file("brown_nouns.txt", "brown_noun_output.txt")

In [6]:
# Pluralizer for all nouns in brown_noun.txt

def pluralize(noun):
    es_endings = ('s', 'z', 'x', 'ch', 'sh')

    # Plural: "ies" ending (Y-replacement rule)
    if noun.endswith('ies'):
        stem = noun[:-3]
        # Singular form would be stem + "y"
        return f"{stem}y+N+PL"

    # Plural: "es" ending (E-insertion rule)
    elif noun.endswith('es'):
        stem = noun[:-2]
        # Accept only if stem ends with ES-ending
        if stem.endswith(es_endings):
            return f"{stem}+N+PL"
        else:
            return "Invalid Word"

    # Plural: regular "s" ending (S addition rule)
    elif noun.endswith('s'):
        # Avoid double-handling "ies" and "es"
        if noun.endswith('ies') or noun.endswith('es'):
            pass  # Already handled above
        stem = noun[:-1]
        # If stem ends with 'y', plural should be 'ies'
        if stem.endswith('y'):
            return "Invalid Word"
        # If stem ends with ES-ending, plural should be 'es'
        if stem.endswith(es_endings):
            return "Invalid Word"
        # Otherwise, valid plural (candidates, signatures, bags, etc.)
        return f"{stem}+N+PL"

    # Singular noun
    else:
        return f"{noun}+N+SG"


def process_file(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            noun = line.strip()
            if not noun:
                continue
            result = pluralize(noun)
            outfile.write(f"{noun} = {result}\n")


if __name__ == "__main__":
    process_file("brown_nouns.txt", "brown_noun_output.txt")

In [7]:
def pluralize(noun):
    es_endings = ('s', 'z', 'x', 'ch', 'sh')

    # Plural: "ies" ending (Y-replacement rule)
    if noun.endswith('ies'):
        stem = noun[:-3]
        return f"{stem}y+N+PL"

    # Plural: "es" ending (E-insertion rule)
    elif noun.endswith('es'):
        stem = noun[:-2]
        if stem.endswith(es_endings):
            return f"{stem}+N+PL"
        else:
            return "Invalid Word"

    # Plural: regular "s" ending (S addition rule)
    elif noun.endswith('s'):
        # Avoid double-handling "ies" and "es"
        if noun.endswith('ies') or noun.endswith('es'):
            return "Invalid Word"

        stem = noun[:-1]
        # Accept as regular plural
        return f"{stem}+N+PL"

    # Singular noun
    else:
        return f"{noun}+N+SG"


def process_file(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            noun = line.strip()
            if not noun:
                continue
            result = pluralize(noun)
            outfile.write(f"{noun} = {result}\n")


if __name__ == "__main__":
    process_file("brown_nouns.txt", "output.txt")
